In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

%matplotlib notebook 

In [3]:
# Puede ser que aparezca un error por no tener instalada la librería ipympl. En ese caso, descomentar la siguiente línea y correrla:
#pip install ipympl

### Método de la potencia

In [4]:
# Implementación del método de la potencia

def metodo_potencia(A, v, tol=1e-6, max_iter=100):
    r_ant = np.inf
    r_k = v @ (A @ v) / (v @ v)
    k = 1
    while k < max_iter and np.abs(r_ant - r_k) > tol:
        r_ant = r_k
        w = A @ v
        v = w / np.linalg.norm(w)
        r_k = v @ (A @ v) / (v @ v)
        k += 1
    if k == max_iter:
        print('Maximo de interaciones alcanzado')
    return r_k, v

### Ejemplo en clase

In [5]:
# Ejemplo 1B)

A = np.array([[-3, 7, -5], [0, 4, 0], [-5, 5, -3]])
v_0 = np.array([5,3,2])
lamb, u = metodo_potencia(A, v_0)
print('Aproximación del autovalor de módulo máximo: ', lamb)
print('Aproximación de su correspondiente autovector: ', u)

Aproximación del autovalor de módulo máximo:  -8.000000268220889
Aproximación de su correspondiente autovector:  [-7.07106765e-01  3.16101371e-08 -7.07106797e-01]


In [6]:
# Ejemplo 1C)

v_0 = 2*np.array([1,1,0]) - 3*np.array([-1, 0, 1])
lamb, u = metodo_potencia(A,v_0)
print('Aproximación del autovalor de módulo máximo: ', lamb)
print('Aproximación de su correspondiente autovector: ', u)

Aproximación del autovalor de módulo máximo:  3.999999284840236
Aproximación de su correspondiente autovector:  [ 7.07107034e-01  7.07106528e-01 -5.05773372e-07]


### ¿Cómo elijo $v^{(0)}$?

En general, si sabemos que $A$ es diagonalizable con único autovalor de módulo máximo, es **muy** probable (de hecho, con probabilidad $1$) que cualquier $v^{(0)}$ que elijamos sirva:

In [7]:
for i in range(100):
    v_0 = 2*np.random.rand(3) - 1  # Vector aleatorio con coordenadas en [-1,1]
    lamb, u = metodo_potencia(A, v_0)
    print('Aproximación del autovalor de módulo máximo: ', lamb)

Aproximación del autovalor de módulo máximo:  -8.000000175879034
Aproximación del autovalor de módulo máximo:  -7.9999998330707465
Aproximación del autovalor de módulo máximo:  -8.000000312930702
Aproximación del autovalor de módulo máximo:  -8.000000193997604
Aproximación del autovalor de módulo máximo:  -7.999999674637097
Aproximación del autovalor de módulo máximo:  -7.999999783617466
Aproximación del autovalor de módulo máximo:  -7.999999696768416
Aproximación del autovalor de módulo máximo:  -7.999999699623093
Aproximación del autovalor de módulo máximo:  -7.9999998117189755
Aproximación del autovalor de módulo máximo:  -7.9999996997581295
Aproximación del autovalor de módulo máximo:  -8.000000274816921
Aproximación del autovalor de módulo máximo:  -7.999999704688033
Aproximación del autovalor de módulo máximo:  -7.999999770801279
Aproximación del autovalor de módulo máximo:  -8.000000188743781
Aproximación del autovalor de módulo máximo:  -7.999999733330307
Aproximación del autov

### Visualización del progreso del método

Para visualizarlo fácilmente, veamos un ejemplo en $\mathbb{R}^2$:

$$A = \begin{pmatrix}
-1 & 3 \\
0 & 2 \\
\end{pmatrix}$$

Sale directo (ejercicio de la guía) que los autovalores son $\lambda_1=2$ y $\lambda_2=-1$. Como $A$ es diagonalizable y tiene único autovalor de módulo máximo, el método converge.
Los autovectores (normalizados) de $A$ son:
$$u_1=\left(\frac{1}{\sqrt{2}},\frac{1}{\sqrt{2}}\right) \quad u_2=(1,0)$$ <br>

In [8]:
A = np.array([[-1, 3], [0, 2]])
# Tomamos un vector aleatorio con coordenadas en [-1,1]
v_0 = 2*np.random.rand(2) - 1

# Aplicamos el método de la potencia y guardamos todas las aproximaciones
v = v_0
max_iter = 100
Vks = [v]
r_ant = np.inf
r_k = v @ (A @ v) / (v @ v)
k = 1
while k < max_iter:
    v_ant = v.copy()  # Guardamos v_(k-1)
    r_ant = r_k
    w = A @ v
    v = w / np.linalg.norm(w)
    r_k = v @ (A @ v) / (v @ v)
    k += 1
    Vks.append(v)

# Acá comienza la parte de graficar: 

fig, ax = plt.subplots()
ax.set_xlim(-1.1, 1.1)
ax.set_ylim(-1.1, 1.1)
ax.set_aspect(1)

line, = ax.plot([])
eigv_approx, = ax.plot([], 'C0*', label=r'Aproximación de $u_1$')

circunferencia = plt.Circle((0,0), 1, color='red', fill=False, linestyle='--', label='Circunferencia unitaria')
ax.add_artist(circunferencia)

origen = plt.plot(0,0,'o', label=r'(0,0)')[0]
ax.add_artist(origen)

# Calculamos los autovectores de A con numpy para graficar <u_1>
eigvals, C = np.linalg.eig(A)
u_1 = C[:, np.argmax(np.abs(eigvals))]

# Graficamos <u_1>
subesp_u1 = plt.axline((0,0), u_1, label=u'$<u_1>$', alpha=0.4)
ax.add_artist(subesp_u1)

def animate(frame):
    v = Vks[frame]
    line.set_data((np.array([0, v[0]]), np.array([0, v[1]])))
    eigv_approx.set_data(*v)
    eigv_approx.set(marker='*', markersize=15, color='C0', label='test')
    return line

anim = FuncAnimation(fig, animate, frames=max_iter, interval=650)

plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3)
plt.show()

<IPython.core.display.Javascript object>

### ¿Qué pasa si le aplicamos el método a una matriz que no tiene único autovalor de  máximo módulo?

Sea:
$$A = \begin{pmatrix}
0 & 0 & -3 \\
1 & 1 & -1 \\
-3 & 0 & 0
\end{pmatrix}$$
se puede calcular que los autovalores y autovectores de $A$ son:
$$\lambda_1=3 \quad \lambda_2=-3 \quad \lambda_3=1$$
$$u_1 = (1, 1, -1) \quad u_2=(1,0,1) \quad u_3=(0,1,0)$$

Veamos qué pasa cuando aplicamos el método:

In [9]:
A = np.array([[0,0,-3], [1,1,-1], [-3,0,0]])
v_0 = 2*np.random.rand(3) - 1
lamb, u = metodo_potencia(A, v_0)
print('Aproximación del autovalor de módulo máximo: ', lamb)
print('Aproximación de su correspondiente autovector: ', u)
print('Autovalores de A: ', np.linalg.eigvals(A))

Aproximación del autovalor de módulo máximo:  2.4104427395031185
Aproximación de su correspondiente autovector:  [-0.76990408 -0.54825219  0.32659951]
Autovalores de A:  [ 1.  3. -3.]


Converge a un valor erróneo. <br>

### Pero si tiene único autovalor de máximo módulo con multiplicidad $\geq1$, funciona

In [10]:
A = np.array([[1, 0, 1], [-1, 2, 1], [1, 0, 1]])
v_0 = 2*np.random.rand(3) - 1
lamb, u = metodo_potencia(A, v_0)
print('Aproximación del autovalor de módulo máximo: ', lamb)
print('Aproximación de su correspondiente autovector: ', u)
print('Autovalores de A: ', np.linalg.eigvals(A))

Aproximación del autovalor de módulo máximo:  2.0
Aproximación de su correspondiente autovector:  [0.67951816 0.27660469 0.67951816]
Autovalores de A:  [2. 2. 0.]


**Obs:** en este caso el método devuelve un vector del autoespacio $E_2$

### Método de la potencia inversa

**Proposición:** sea $A \in \mathbb{C}^{n\times n}$ inversible, si $\lambda$ es autovalor de $A$, entonces $\frac{1}{\lambda}$ es autovalor de A con el mismo autovector.

**Idea:** si aplico el método de la potencia a $A^{-1}$, obtengo $\mu$ su autovalor de módulo máximo. Como $\mu=\frac{1}{\lambda}$ para $\lambda$ algún autovalor de $A$, significa que $\lambda$ es de módulo mínimo.

**Obs:** ahora tenemos que inicializar con $v^{(0)}$ que no tenga coordenada en $u_n$ nula.

In [11]:
# Implementación del método de la potencia

def metodo_potencia_inversa(A, v, tol=1e-6, max_iter=100):
    r_ant = np.inf
    Ainv_v = np.linalg.solve(A, v) # Con esto evitamos tener que invertir A para calcular A^(-1)v
    r_k = v @ Ainv_v / (v @ v)
    k = 1
    while k < max_iter and np.abs(r_ant - r_k) > tol:
        r_ant = r_k
        w = Ainv_v
        v = w / np.linalg.norm(w)
        Ainv_v = np.linalg.solve(A, v)
        r_k = v @ Ainv_v / (v @ v)
        k += 1
    if k == max_iter:
        print('Maximo de interaciones alcanzado')
    return 1/r_k, v  # Devolvemos 1/r_k pues r_k es el autovalor de la inversa de A

In [12]:
# Aplicándolo a la matriz del ejemplo visto en clase 

A = np.array([[-3, 7, -5], [0, 4, 0], [-5, 5, -3]])
v_0 = np.array([-1, -1,  4])
lamb, u = metodo_potencia_inversa(A, v_0)
# lamb, u = metodo_potencia(np.linalg.inv(A), v_0)
print('Aproximación del autovalor de módulo mínimo: ', lamb)
print('Aproximación de su correspondiente autovector: ', u)

Aproximación del autovalor de módulo mínimo:  2.000003814733642
Aproximación de su correspondiente autovector:  [-7.07109478e-01 -5.39477603e-06  7.07104084e-01]


### ¿Y si quiero todos los autovalores?

Bajo las hipótesis del teorema, sean $B=\{u_1,\dots,u_n\}$ los autovectores normalizados de $A$ (en orden descendiente según el módulo de sus autovalores correspondientes), entonces se puede ver que:

$$ A = CDC^{-1} = \displaystyle\sum_{i=1}^n \lambda_iu_iu_i^t$$

**Idea:** Iterativamente vamos "eliminando" el autovector correspondiente al autovalor de módulo máximo:

$A_1 = A$ <br>
$\lambda_1, \hat{u}_1$ = ``metodo_potencia(``$A_1$ ``, v_0)``<br>
for $k=2,\dots,n$:<br>
&nbsp;&nbsp;&nbsp;&nbsp; $A_k = A_{(k-1)} - \lambda_{k-1}\hat{u}_{k-1}\hat{u}_{k-1}^t$<br>
&nbsp;&nbsp;&nbsp;&nbsp; $\lambda_k, \hat{u}_k$ = ``metodo_potencia(``$A_{k}$ ``, v_0)``<br>

**Obs:** $v^{(0)}$ no debería tener ninguna coordenada nula en $B=\{u_1,\dots,u_n\}$ <br>
**Obs:** en general $\hat{u}_k$ no son los autovectores de $A$

En la materia veremos otros métodos que nos permitirán también recuperar los autovectores.

In [13]:
def encontrar_autovalores(A, v_0, tol=1e-6):
    autovals = []
    n = A.shape[0]
    u = np.zeros(n)
    lamb = 0
    for k in range(n):
        A = A - lamb*(np.outer(u,u))
        lamb, u = metodo_potencia(A, v_0, tol)
        autovals.append(lamb)
    return autovals

In [14]:
# Esta matriz tiene autovalores {-8, 4, 2}
A = np.array([[-3, 7, -5], [0, 4, 0], [-5, 5, -3]])
v_0 = np.random.rand(3)
autovals = encontrar_autovalores(A, v_0)
print('Autovalores de A: ', autovals)

Autovalores de A:  [-8.000000323217204, 4.000000063569689, 1.999998579990913]


In [15]:
# Esta matriz tiene autovalores {2, -1}
A = np.array([[-1, 3], [0, 2]])
v_0 = np.random.rand(2)
autovals = encontrar_autovalores(A, v_0)
print('Autovalores de A: ', autovals)

Autovalores de A:  [1.9999998309292384, -1.0000003381413232]


In [16]:
# Esta matriz tiene autovalores {2,2,0}
A = np.array([[1, 0, 1], [-1, 2, 1], [1, 0, 1]])
v_0 = 2*np.random.rand(3) - 1
autovals = encontrar_autovalores(A, v_0)
print('Autovalores de A: ', autovals)

Autovalores de A:  [2.0, 2.0, -2.255364136002869e-16]
